In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import copy
import random

2023-09-26 19:07:22.518027: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 19:07:22.988845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 1024
learning_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
class ParaServer:
    def __init__(self):
        self.model = MLP()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.freqs = {}
    def upload(self, grads, freqs, score):
        for freq in freqs:
            self.freqs[freq] = max(0, score)
        self.optimizer.apply_gradients(grads_and_vars=zip(grads, self.model.variables))
        return self.model, self.freqs
    def download(self):
        return self.model, self.freqs
    def initModel(self, x):
        self.model(x)
    def lr_decay(self, ratio):
        self.optimizer.learning_rate = self.optimizer.learning_rate * ratio

In [5]:
def valiAll(index_epoch):
    model, _ = ps.download()
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))
    r2sv[index_epoch] = va_r2.numpy()

In [6]:
class Node:
    def __init__(self, id, freq):
        self.id = id
        self.freq = freq
        self.otfreqs = {}
        self.model = MLP()
        self.zeroModel = MLP()
        self.dataset1 = pd.read_csv('./20-24Trainset.csv', encoding='utf-8')
        self.dataset2 = pd.read_csv('./50-54Trainset.csv', encoding='utf-8')
        self.dataset = pd.concat([self.dataset1, self.dataset2], axis=0).sample(frac=1).reset_index(drop=True)
        self.dataset = self.dataset[self.dataset['freq'].isin(self.freq)]
        self.X = self.dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
        self.y = self.dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
        self.dataset_train = tf.data.Dataset.from_tensor_slices((self.X, self.y))
        self.dataset_train = self.dataset_train.shuffle(buffer_size=self.X.shape[0])
        self.dataset_train = self.dataset_train.batch(batch_size)
        self.dataset_train = self.dataset_train.prefetch(tf.data.experimental.AUTOTUNE)
    def getZero(self):
        m, freqs = ps.download()
        self.otfreqs = copy.deepcopy(freqs)
        self.zeroModel = copy.deepcopy(m)
    def train(self, index_epoch):
        self.model, _ = ps.download()
        for X, y in self.dataset_train:
            with tf.GradientTape() as tape:
                y_pred = self.model(X)
                tr_mse = tf.reduce_mean(tf.square(y_pred - y))
            tr_rmse = tf.sqrt(tr_mse)
            tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
            tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
            grads = tape.gradient(tr_mse, self.model.variables)
            sum_r2 = 1
            for k, v in self.otfreqs.items():
                if k in self.freq or v == 0:
                    continue
                X_i = tf.tensor_scatter_nd_update(X, [[i, 0] for i in range(X.shape[0])], [k] * X.shape[0])
                y_i = self.zeroModel(X_i)
                with tf.GradientTape() as tape:
                    y_pred_i = self.model(X_i)
                    loss = tf.reduce_mean(tf.square(y_pred_i - y_i))
                grad = tape.gradient(loss, self.model.variables)
                grads = [grads[i] + grad[i] * v for i in range(len(grads))]
                sum_r2 += v
            self.model, _ = ps.upload([i / sum_r2 for i in grads], self.freq, tr_r2.numpy())
        # if epoch_index in np.arange(0, num_epochs, 25).tolist() or epoch_index == num_epochs - 1:
        print("node:{} epoch:{}".format(self.freq, index_epoch))
        print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
        r2s[self.id][index_epoch] = tr_r2.numpy()

In [7]:
r2s = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}}
r2sv = {}

In [8]:
test_dataset = pd.read_csv("testset.csv", encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [9]:
ps = ParaServer()
ps.initModel(X_v)

2023-09-26 19:07:24.618494: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-09-26 19:07:24.619057: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [10]:
nodeList = [Node(0, [2.0, 5.0]), Node(1, [2.1, 5.1]), Node(2, [2.2, 5.2]), Node(3, [2.3, 5.3]), Node(4, [2.4, 5.4])]

In [11]:
orders = [0, 1, 2, 3, 4]
turn = [np.array([[92, 146], [158, 255], [347, 475], [531, 555]]), 
        np.array([[42, 116], [226, 277], [363, 423], [543, 600]]), 
        np.array([[0, 200], [214, 252], [271, 347], [474, 528]]),
        np.array([[68, 132], [173, 305], [418, 423], [563, 587]]),
        np.array([[7, 151], [216, 305], [357, 360], [420, 538]]),]
for i in range(600):
    random.shuffle(orders)
    for j in orders:
        for l, r in turn[j]:
            if l <= i < r:
                if l == i:
                    nodeList[j].getZero()
                nodeList[j].train(i)
    valiAll(i)
    if i in [100, 200, 300, 400, 500]:
        ps.lr_decay(0.7)

2023-09-26 19:07:27.370974: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562706be6b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 19:07:27.370996: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 19:07:27.371000: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-26 19:07:27.374150: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 19:07:27.480122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-26 19:07:27.585796: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


node:[2.2, 5.2] epoch:0
train mse:0.07112941145896912 rmse:0.26670098304748535 mae:0.2072775661945343 r2:0.4180770516395569
mse:0.07679049670696259 rmse:0.2771109640598297 mae:0.21703149378299713 r2:0.3710636496543884
node:[2.2, 5.2] epoch:1
train mse:0.06915976107120514 rmse:0.26298242807388306 mae:0.20760183036327362 r2:0.4327087998390198
mse:0.072026826441288 rmse:0.2683781385421753 mae:0.21010825037956238 r2:0.4100794792175293
node:[2.2, 5.2] epoch:2
train mse:0.06217111274600029 rmse:0.24934135377407074 mae:0.19527970254421234 r2:0.4914523959159851
mse:0.06501331180334091 rmse:0.2549770772457123 mae:0.20010054111480713 r2:0.46752220392227173
node:[2.2, 5.2] epoch:3
train mse:0.05350400134921074 rmse:0.23130932450294495 mae:0.17917606234550476 r2:0.562127411365509
mse:0.059227317571640015 rmse:0.24336662888526917 mae:0.1902114301919937 r2:0.5149111747741699
node:[2.2, 5.2] epoch:4
train mse:0.051165368407964706 rmse:0.22619763016700745 mae:0.17721596360206604 r2:0.5804755687713623


In [19]:
for k, v in r2sv.items():
    print(v)

0.37106365
0.41007948
0.4675222
0.5149112
0.55359197
0.5783335
0.5746819
0.59190583
0.60767716
0.6069933
0.6498928
0.62810886
0.65168905
0.6665416
0.6757158
0.6658553
0.64663136
0.64593875
0.6683818
0.6867815
0.649281
0.6710172
0.69861853
0.6513511
0.6952268
0.6569839
0.7070579
0.7100551
0.7155792
0.6944573
0.66306114
0.6732335
0.676183
0.6539908
0.71198726
0.7194927
0.7193106
0.6816006
0.6784838
0.63468724
0.7323679
0.7272787
0.74898225
0.69079393
0.7340168
0.6603653
0.7665893
0.7149126
0.77107036
0.7152362
0.7077371
0.7479875
0.7523867
0.7553145
0.7500777
0.75842255
0.75413156
0.7264737
0.7230927
0.7284945
0.7133199
0.79192805
0.79508364
0.7639031
0.7712383
0.7698622
0.7645274
0.8001182
0.7407745
0.73693335
0.80310005
0.80273175
0.79690284
0.73565066
0.78101754
0.7983575
0.7917686
0.789534
0.73209584
0.74141026
0.7954948
0.8134084
0.7963808
0.79588693
0.81382966
0.73358274
0.7496158
0.7500895
0.7324647
0.81151855
0.80041367
0.8143796
0.76598006
0.7469182
0.7935949
0.83205837
0.799686

In [13]:
r2s

{0: {92: 0.84553385,
  93: 0.8222211,
  94: 0.86392033,
  95: 0.82405114,
  96: 0.8686048,
  97: 0.8294425,
  98: 0.8446878,
  99: 0.8444689,
  100: 0.8391388,
  101: 0.86540544,
  102: 0.864836,
  103: 0.885788,
  104: 0.85915303,
  105: 0.87829715,
  106: 0.8873132,
  107: 0.870481,
  108: 0.85874635,
  109: 0.8788102,
  110: 0.8718687,
  111: 0.85762286,
  112: 0.8730758,
  113: 0.8693012,
  114: 0.8946666,
  115: 0.8695388,
  116: 0.88058054,
  117: 0.881482,
  118: 0.91447574,
  119: 0.87904793,
  120: 0.8859246,
  121: 0.8766124,
  122: 0.8909323,
  123: 0.91793525,
  124: 0.8904761,
  125: 0.8842548,
  126: 0.88262117,
  127: 0.8947408,
  128: 0.8769933,
  129: 0.8887103,
  130: 0.8841338,
  131: 0.88009715,
  132: 0.8964255,
  133: 0.901202,
  134: 0.90298665,
  135: 0.9017139,
  136: 0.8884151,
  137: 0.90137476,
  138: 0.8864537,
  139: 0.89356935,
  140: 0.89654744,
  141: 0.88912755,
  142: 0.8968313,
  143: 0.8927078,
  144: 0.912034,
  145: 0.90724516,
  158: 0.87910175,
